# Feature extraction

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import feature_extraction

## Load dataset

In [3]:
# Load CB16 dataset
df = pd.read_json("../data/clickbait17-train-170331/instances.jsonl", lines=True, encoding='utf8')
df.set_index("id", inplace=True)
df_raw = df.copy()

df_truth = pd.read_json("../data/clickbait17-train-170331/truth.jsonl", lines=True, encoding='utf8')
df_truth.set_index("id", inplace=True)
df_truth_raw = df_truth.copy()

In [4]:
# Add truth judgments to df
df = df.join(df_truth[["truthClass"]]) # also available: truthClass, truthJudgments, truthMean, truthMedian, truthMode

In [5]:
df.head(2)

,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,truthClass
id,,,,,,,,,
608310377143799808,[],[Apple's iOS 9 'App thinning' feature will giv...,Tue Jun 09 16:31:10 +0000 2015,['App thinning' will be supported on Apple's i...,'App thinning' will be supported on Apple's iO...,"Apple,gives,gigabytes,iOS,9,app,thinning,featu...",[Paying for a 64GB phone only to discover that...,Apple gives back gigabytes: iOS 9 'app thinnin...,no-clickbait
609297109095972864,[media/609297109095972864.jpg],[RT @kenbrown12: Emerging market investors are...,Fri Jun 12 09:52:05 +0000 2015,"[Stocks Fall as Investors Watch Central Banks,...",Global investors have yanked $9.3 billion from...,"emerging market,emerging markets,em flows,em i...","[Emerging markets are out of favor., Global in...",Emerging Markets Suffer Largest Outflow in Sev...,no-clickbait


## Extract features

In [7]:
fe = feature_extraction.FeatureExtractor(df.head(2), "../data/clickbait17-train-170331/")
labels, features = fe.extract_features()

C:\Users\kaspe\Anaconda3\envs\ir\lib\site-packages\PyDictionary\utils.py:5: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 5 of the file C:\Users\kaspe\Anaconda3\envs\ir\lib\site-packages\PyDictionary\utils.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  return BeautifulSoup(requests.get(url).text)


Error: The Following Error occured: list index out of range
Error: The Following Error occured: list index out of range
Error: The Following Error occured: list index out of range
Error: The Following Error occured: list index out of range
Error: The Following Error occured: list index out of range
Error: The Following Error occured: list index out of range
Error: The Following Error occured: list index out of range
Error: The Following Error occured: list index out of range
Error: The Following Error occured: list index out of range
Error: The Following Error occured: list index out of range
Error: The Following Error occured: list index out of range
Error: The Following Error occured: list index out of range


In [8]:
features.sample(2)

,numChars_PostTitle,ratioChars_PostImagePostTitle,diffChars_PostTitleArticleKeywords,diffChars_PostTitlePostImage,ratioWords_PostImagePostTitle,numWords_PostTitle,numFormalWords_PostTitle,ratioWords_ArticleDescPostTitle,ratioChars_ArticleDescPostTitle,ratioChars_ArticleTitlePostTitle,ratioWords_ArticleTitlePostTitle,diffWords_PostTitleArticleKeywords,numQuestionmarksPostTitle,ratioChars_ArticleParagraphsPostTitle,ratioChars_ArticleParagraphsArticleDesc
id,,,,,,,,,,,,,,,
608310377143799808,63.0,-1.000000,15.0,-1.0,-1.0,13.0,9.0,18.0,2.444444,1.380952,1.307692,0.0,0.0,1.576564,0.644958
609297109095972864,85.0,2.352941,35.0,115.0,2.4,15.0,10.0,12.0,1.647059,0.552941,0.533333,4.0,0.0,1.427451,0.866667
